**Import required libraries and dependencies:**

**Required: tmdb API Key (sign-up at tmdb website)**

In [ ]:
import tmdbsimple as tmdb
import requests
import pandas as pd
tmdb.API_KEY = '9bf602d556fc87283e2577bc183b8492'
tmdb.REQUESTS_TIMEOUT = 5  # seconds, for both connect and read

**Defined a list for each columns/fields to store the values when scraping:**

In [7]:
listNames=[]
listReviews=[]
listOverallRatings=[]
listUserRatings=[]
listGenre=[]
listId=[]
listIsVideo=[]
listReleaseDate=[]
listProductionCompany=[]
text_query = ['a','e','i','o','r']
max_page=200


#
**Testing tmdb search endpoint but found a few limitations:**
1. Unknown how many of the movies returned has reviews
2. Possibility of overlaps in the different search results

In [132]:
all_search_results=[]
#42.7sec
#Explore Search using boolean text query
search = tmdb.Search()
for i in range(1,max_page+1):
    for letter in text_query:
        try:
            response = search.movie(query=letter,page=i)
            for result in search.results:
                all_search_results.append(result)
        except IndexError:
            pass
print(len(all_search_results))
#print(len(search.results))

20000


In [19]:
#Get Title
print(tmdb.Movies(id="209500").info()["title"])
#Get Genre and put into list
#print(tmdb.Movies(id="2").info()["genres"])
# genre_string = ""
# current_movie_genre = tmdb.Movies(id="2").info()["genres"]
# #Logic for retrieving genres
# for i in range(len(current_movie_genre)):
#     if i!=len(current_movie_genre)-1:
#         genre_string+=f"{current_movie_genre[i]['name']},"
#     else:
#         genre_string+=f"{current_movie_genre[i]['name']}"
# print(current_movie_genre)
# print(genre_string)

#Get tmdbRating
#print(tmdb.Movies(id="2").info()['production_companies'][0]['name'])

#print(tmdb.Movies(id='640146').reviews()["results"][0])

# for review in tmdb.Movies(id="209478").reviews()["results"]:
#     print(review['author_details']['rating'])
#Get First Review
#print(len())

#Getting the content of the review
#Inside reviews -> results(lists) -> content ->
#print(tmdb.Movies(id="198663").reviews()["results"][0]['content'].replace('\r\n\r',' ').replace('\n',' '))

#Getting the rating of the review
#print(tmdb.Movies(id="198663").reviews()["results"][0]['author_details']['rating'])

Uncommon Women and Others


**Below is the logic for getting the desired movie data from tmdb movie_review endpoint:**
1. We continuously iterate through the movie_ids starting from 0,
2. Checking whether the movie has reviews and whether the reviews have user_ratings assigned to them
3. If it has, we have 3 scenarios
4. We check whether we have reached the desired number of good/bad reviews else, we will retrieve the respective data from the response and store into their individual list

In [83]:
#8.2sec for 100
good_reviews=543
bad_reviews=3496
count=9998
max_count=10000
idx=353442
while count!=max_count:
    if count%10==0:
        print(f"Count: {count}")
    try:
        result_id = idx
        #If got reviews, extract details
        if(good_reviews!=0 and bad_reviews!=0):
            #Check if movie has reviews
            if(len(tmdb.Movies(id=result_id).reviews()["results"])!=0):
                #Check each reviews if they have ratings
                for review in tmdb.Movies(id=result_id).reviews()["results"]:
                    if (review['author_details']['rating']!= None):
                        #Retrieve id, name, review, userRating(user given), tmdbRating(avg of all user ratings), genre, isVideoTag, releaseDate, ProductionCompany
                        listNames.append(tmdb.Movies(id=result_id).info()['title'])
                        listIsVideo.append(tmdb.Movies(id=result_id).info()['video'])
                        listReleaseDate.append(tmdb.Movies(id=result_id).info()['release_date'])
                        if(len(tmdb.Movies(id=result_id).info()['production_companies'])!=0):
                            listProductionCompany.append(tmdb.Movies(id=result_id).info()['production_companies'][0]['name'])
                        else:
                            listProductionCompany.append("Unknown")
                        listId.append(result_id)
                        listOverallRatings.append(tmdb.Movies(id=result_id).info()['vote_average'])
                        listReviews.append(review['content'].replace('\r\n\r',' ').replace('\n',' '))
                        listUserRatings.append(review['author_details']['rating'])
                        #Logic to retrieve genres of the movie and clean and convert to a string
                        genre_string = ""
                        current_movie_genre = tmdb.Movies(id=result_id).info()["genres"]
                        #Logic for retrieving genres
                        for i in range(len(current_movie_genre)):
                            if i!=len(current_movie_genre)-1:
                                genre_string+=f"{current_movie_genre[i]['name']},"
                            else:
                                genre_string+=f"{current_movie_genre[i]['name']}"
                        listGenre.append(genre_string)
                        genre_string=""
                        #Logic to check if the review rating is good or bad (given that we take >=5 as good)
                        if (review['author_details']['rating']>=5):
                            good_reviews-=1
                        else:
                            bad_reviews-=1
                        count+=1
                        if count==max_count:
                            break
        elif(good_reviews==0):
            #Check if movie has reviews
            if(len(tmdb.Movies(id=result_id).reviews()["results"])!=0):
                #Check each reviews if they have ratings
                for review in tmdb.Movies(id=result_id).reviews()["results"]:
                    if (review['author_details']['rating']!= None and review['author_details']['rating']<5):
                        #Retrieve id, name, review, userRating(user given), tmdbRating(avg of all user ratings), genre, isVideoTag, releaseDate, ProductionCompany
                        listNames.append(tmdb.Movies(id=result_id).info()['title'])
                        listIsVideo.append(tmdb.Movies(id=result_id).info()['video'])
                        listReleaseDate.append(tmdb.Movies(id=result_id).info()['release_date'])
                        if(len(tmdb.Movies(id=result_id).info()['production_companies'])!=0):
                            listProductionCompany.append(tmdb.Movies(id=result_id).info()['production_companies'][0]['name'])
                        else:
                            listProductionCompany.append("Unknown")
                        listId.append(result_id)
                        listOverallRatings.append(tmdb.Movies(id=result_id).info()['vote_average'])
                        listReviews.append(review['content'].replace('\r\n\r',' ').replace('\n',' '))
                        listUserRatings.append(review['author_details']['rating'])
                        #Logic to retrieve genres of the movie and clean and convert to a string
                        genre_string = ""
                        current_movie_genre = tmdb.Movies(id=result_id).info()["genres"]
                        #Logic for retrieving genres
                        for i in range(len(current_movie_genre)):
                            if i!=len(current_movie_genre)-1:
                                genre_string+=f"{current_movie_genre[i]['name']},"
                            else:
                                genre_string+=f"{current_movie_genre[i]['name']}"
                        listGenre.append(genre_string)
                        genre_string=""
                        
                        
                        bad_reviews-=1
                        
                        count+=1
                        if count==max_count:
                            break
        elif(bad_reviews==0):
            if(len(tmdb.Movies(id=result_id).reviews()["results"])!=0):
                #Check each reviews if they have ratings
                for review in tmdb.Movies(id=result_id).reviews()["results"]:
                    if (review['author_details']['rating']!= None and review['author_details']['rating']>=5):
                        #Retrieve id, name, review, userRating(user given), tmdbRating(avg of all user ratings), genre, isVideoTag, releaseDate, ProductionCompany
                        listNames.append(tmdb.Movies(id=result_id).info()['title'])
                        listIsVideo.append(tmdb.Movies(id=result_id).info()['video'])
                        listReleaseDate.append(tmdb.Movies(id=result_id).info()['release_date'])
                        if(len(tmdb.Movies(id=result_id).info()['production_companies'])!=0):
                            listProductionCompany.append(tmdb.Movies(id=result_id).info()['production_companies'][0]['name'])
                        else:
                            listProductionCompany.append("Unknown")
                        listId.append(result_id)
                        listOverallRatings.append(tmdb.Movies(id=result_id).info()['vote_average'])
                        listReviews.append(tmdb.Movies(id=result_id).reviews()["results"][0]['content'].replace('\r\n\r',' ').replace('\n',' '))
                        listUserRatings.append(tmdb.Movies(id=result_id).reviews()["results"][0]['author_details']['rating'])
                        #Logic to retrieve genres of the movie and clean and convert to a string
                        genre_string = ""
                        current_movie_genre = tmdb.Movies(id=result_id).info()["genres"]
                        #Logic for retrieving genres
                        for i in range(len(current_movie_genre)):
                            if i!=len(current_movie_genre)-1:
                                genre_string+=f"{current_movie_genre[i]['name']},"
                            else:
                                genre_string+=f"{current_movie_genre[i]['name']}"
                        listGenre.append(genre_string)
                        genre_string=""
                        
                        good_reviews-=1
                        
                        count+=1
                        if count==max_count:
                            break
    except requests.exceptions.HTTPError as err:
        pass
    idx+=1

**For tracking/debug:**

In [84]:
# print(count) #10000 Entries now
# print(good_reviews) # 541
# print(bad_reviews) #3496
# print(idx) #Stopped at 353487
# print(listGenre[0])
# print(tmdb.Movies(id=82880).info()["genres"])


10000
541
3496
353487
Thriller,Horror
[]


In [85]:
#print(listNames)
print(len(listReviews))
print(len(listOverallRatings))
print(len(listUserRatings))
print(len(listNames))#
print(len(listId))
print(len(listGenre))
print(len(listIsVideo))#
print(len(listReleaseDate))#
print(len(listProductionCompany))
print(good_reviews)
print(count)

4238
4238
4238
4238
4238
4238
4238
4238
4238
541
10000


**Creating a pd.df with scraped data and save as CSV**

In [102]:
data = {
    'Movie Name': listNames,
    'Movie Id' : listId,
    'Review Content': listReviews,
    'tmdb Rating' : listOverallRatings,
    'User Rating': listUserRatings,
    'Genre(s)' : listGenre,
    'Not A Movie?' : listIsVideo,
    'Release Date' : listReleaseDate,
    'Production Company' : listProductionCompany
}

# Create a Pandas dataframe from the data
df = pd.DataFrame(data)

# Save the dataframe to a CSV file
#df.to_csv('test_clean_data_extended_newest_data.csv', index=True)

None
                          Movie Name  Movie Id  \
4233                           Wagah    353415   
4234              Killing Hasselhoff    353440   
4235              Killing Hasselhoff    353440   
4236                The Magic Canvas    353478   
4237  Jumanji: Welcome to the Jungle    353486   

                                         Review Content  tmdb Rating  \
4233  In his previous film, Haridas, GNR Kumaravelan...        4.400   
4234  Not good, but better than I had prepared mysel...        4.700   
4235  Good watch, would watch again, and can recomme...        4.700   
4236  The opening slide gives us an idea as to what ...        7.000   
4237  **The 90's Game is back and got even more adve...        6.824   

      User Rating                         Genre(s)  Not A Movie? Release Date  \
4233          3.0             Action,Drama,Romance         False   2016-08-12   
4234          4.0                           Comedy         False   2017-08-29   
4235          6.0 

In [192]:
#drop duplicates
pd.unique(df['Movie Name']).size


2331

In [203]:
df_unique = df.drop_duplicates(subset=['Movie Name'])

In [225]:
len(df_unique.index)
df_unique = df_unique.reset_index(drop=True)


In [226]:
df_unique['Movie Name'].nunique()

2331

In [206]:
len(df.index)

2945

In [214]:
df_unique.shape

(2331, 9)

In [228]:
df_unique.to_csv('unique_data.csv',index=True)

In [227]:
df_unique.tail()

,Movie Name,Movie Id,Review Content,tmdb Rating,User Rating,Genre(s),Not A Movie?,Release Date,Production Company
2326,After the Dark,198287,"I've watched a lot of dumb movies, but never h...",6.099,1.0,"Drama,Fantasy,Science Fiction,Thriller",False,2013-07-06,An Olive Branch Productions
2327,American Honey,340485,**To redefine the life and to discover the lov...,6.798,7.0,Drama,False,2016-09-30,BFI
2328,Dead Again,11498,"By far, my favourite film of Branagh's that he...",6.453,9.0,"Thriller,Romance,Mystery",False,1991-08-23,Paramount
2329,April Fool's Day,24913,Muffy the ham-pie slayer! A group of college ...,6.045,7.0,"Horror,Mystery",False,1986-03-27,YCTM
2330,Obsessed,17335,"**Obsessed has a weak, overused concept but a ...",5.747,6.0,"Drama,Thriller",False,2009-04-24,Screen Gems


In [220]:
df_unique.reset_index()

,index,Movie Name,Movie Id,Review Content,tmdb Rating,User Rating,Genre(s),Not A Movie?,Release Date,Production Company
0,0,Winnie the Pooh: Blood and Honey,980078,Now don't judge. You mustn't judge. If you do ...,5.789,5.0,"Horror,Thriller",False,2023-01-27,Jagged Edge Productions
1,1,Avatar: The Way of Water,76600,**Avatar: The Way of Water follows in its pred...,7.728,6.0,"Science Fiction,Adventure,Action",False,2022-12-14,20th Century Studios
2,2,Knock at the Cabin,631842,FULL SPOILER-FREE REVIEW @ https://www.msbrevi...,6.400,8.0,"Horror,Mystery,Thriller",False,2023-02-01,Blinding Edge Pictures
3,3,Black Adam,436270,FULL SPOILER-FREE REVIEW @ https://www.msbrevi...,7.100,5.0,"Action,Fantasy,Science Fiction",False,2022-10-19,New Line Cinema
4,4,A Man Called Otto,937278,"Fans of the BBC sitcom ""One Foot in the Grave""...",7.884,7.0,"Comedy,Drama",False,2022-12-28,Playtone
...,...,...,...,...,...,...,...,...,...,...
2326,2939,After the Dark,198287,"I've watched a lot of dumb movies, but never h...",6.099,1.0,"Drama,Fantasy,Science Fiction,Thriller",False,2013-07-06,An Olive Branch Productions
2327,2940,American Honey,340485,**To redefine the life and to discover the lov...,6.798,7.0,Drama,False,2016-09-30,BFI
2328,2941,Dead Again,11498,"By far, my favourite film of Branagh's that he...",6.453,9.0,"Thriller,Romance,Mystery",False,1991-08-23,Paramount
2329,2942,April Fool's Day,24913,Muffy the ham-pie slayer! A group of college ...,6.045,7.0,"Horror,Mystery",False,1986-03-27,YCTM
